In [15]:
from numpy import *

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import bqplot as bq
import bqplot.interacts as bqi
import ipywidgets as ipw
import ipyvolume as ipv

import traitlets as tr

#Set font
font = {'family':'serif','size':10}
mpl.rc('font',**font)

In [16]:
import h5py

In [47]:
def num_from_spacing(xmin, xmax, dx):
    return floor((xmax - xmin) / dx)
def spacing_from_num(xmin, xmax, nx):
    return (xmax - xmin) / nx

# Initialization

In [18]:
def init_grid(grid):
    grid.xmin, grid.xmax, grid.dx = -1, 1, 5e-2
    grid.ymin, grid.ymax, grid.dy = -1, 1, 5e-2
    grid.zmin, grid.zmax, grid.dz = 0, 10, 1
    grid.ntheta = 20
    grid.nphi = 10
    
    grid.nx = num_from_spacing(grid.xmin, grid.xmax, grid.dx)
    grid.ny = num_from_spacing(grid.ymin, grid.ymax, grid.dy)
    grid.nz = num_from_spacing(grid.zmin, grid.zmax, grid.dz)
    
    grid.x = arange(grid.xmin, grid.xmax, grid.dx)
    grid.y = arange(grid.ymin, grid.ymax, grid.dy)
    grid.z = arange(grid.zmin, grid.zmax, grid.dz)
    
def init_rope(rope, grid):
    a = 2e-1
    b = 1e-1
    c = 5e-1
    
    z = grid.z
    
    rope.kelp_lengths = exp(-a*z) * sin(z) ** 2
    rope.kelp_stds = b * ones_like(z)
    rope.water_speeds = c * ones_like(z)
    rope.water_angles = 2*pi / grid.zmax * z
    
def init_frond(frond):
    frond.fs = .5
    frond.fr = 2
    
def init_params(params):
    params.quadrature_degree = 5

# def init_kelp(kelp, grid):
#     kelp.p_kelp = zeros([grid.nx, grid.ny, grid.nz])

# HDF Read/Write Functions

In [19]:
def hdf_write_trait(hdf, obj, dset_name, dims, dtype):
    try:
        dset = hdf.create_dataset(dset_name, dims, dtype)
    except RuntimeError:
        dset = hdf.require_dataset(dset_name, dims, dtype)
        
    dset[...] = getattr(obj, dset_name)

In [51]:
def hdf_write_grid(filename, grid):
    grid.bounds = (grid.xmin, grid.xmax, grid.ymin, grid.ymax, grid.zmin, grid.zmax)
    grid.nums = (grid.nx, grid.ny, grid.nz, grid.ntheta, grid.nphi)
    #grid.spacings = (grid.dx, grid.dy, grid.dz, grid.dtheta, grid.dphi)
    
    with h5py.File(filename) as file:
        hdf_write_trait(file, grid, 'bounds', (6,), 'd')
        hdf_write_trait(file, grid, 'nums', (5,), 'i')
        #hdf_write_trait(file, grid, 'spacings', (5,), 'd')
        
def hdf_write_rope(filename, rope, grid):
    nz = grid.nz
    
    with h5py.File(filename) as file:
        hdf_write_trait(file, rope, 'kelp_lengths', (nz,), 'd')
        hdf_write_trait(file, rope, 'kelp_stds', (nz,), 'd')
        hdf_write_trait(file, rope, 'water_speeds', (nz,), 'd')
        hdf_write_trait(file, rope, 'water_angles', (nz,), 'd')
    
def hdf_write_frond(filename, frond):
    frond.frond_arr = (frond.fs, frond.fr)
    with h5py.File(filename) as file:
        hdf_write_trait(file, frond, 'frond_arr', (2,), 'd')
        
def hdf_write_params(filename, params):
    params.param_arr = (params.quadrature_degree,)
    with h5py.File(filename) as file:
        hdf_write_trait(file, params, 'param_arr', (1,), 'd')

In [38]:
def hdf_read_kelp(filename, kelp, grid):
    with h5py.File(filename) as file:
        dset = file.require_dataset('param_arr', (grid.nx, grid.ny, grid.nz), 'd')
        kelp.p_kelp = dset[...]

In [39]:
def write_all(grid, rope, frond, params):
    gridfile = '../hdf5/kelp3d/grid.hdf5'  
    ropefile = '../hdf5/kelp3d/rope.hdf5'  
    frondfile = '../hdf5/kelp3d/frond.hdf5'
    paramfile = '../hdf5/kelp3d/param.hdf5' 
    
    hdf_write_grid(gridfile, grid)
    hdf_write_rope(ropefile, rope, grid)
    hdf_write_frond(frondfile, frond)
    hdf_write_params(paramfile, params)
    
def read_all(p_kelp, grid):
    kelpfile = '../hdf5/kelp3d/kelp.hdf5'  
    
    hdf_read_kelp(kelpfile, kelp, grid)

# Create Objects & Assign Traits

In [40]:
grid = tr.HasTraits()

rope = tr.HasTraits()
rope.add_traits(kelp_lengths=tr.Any)
rope.add_traits(kelp_stds=tr.Any)
rope.add_traits(water_speeds=tr.Any)
rope.add_traits(water_angles=tr.Any)

frond = tr.HasTraits()

params = tr.HasTraits()

kelp = tr.HasTraits()

In [41]:
init_grid(grid)
init_rope(rope, grid)
init_frond(frond)
init_params(params)
# init_kelp(kelp)

# Widgets

In [42]:
# Define variables over depth

z_quants = ['vw', 'theta_w_rad', 'L_mean', 'L_std']

z_scale = bq.LinearScale(min=grid.zmin, max=grid.zmax)
z_ax = bq.Axis(scale=z_scale, label='Depth (z)', grid_lines='none')

mins = {
    'vw': 0,
    'theta_w_rad': 0,
    'L_mean': 0,
    'L_std': 0
}

maxs = {
    'vw': 1,
    'theta_w_rad': 2*pi,
    'L_mean': 1,
    'L_std': 1
}

colors = {
    'vw': 'red',
    'theta_w_rad': 'green',
    'L_mean': 'blue',
    'L_std': 'yellow'
}

labels = {
    'vw': 'Water current velocity',
    'theta_w_rad': 'Water current angle',
    'L_mean': 'Frond length mean',
    'L_std': 'Frond length std. dev.'
}


values = {}

values['L_mean'] = rope.kelp_lengths
values['L_std'] = rope.kelp_stds
values['vw'] = rope.water_speeds
values['theta_w_rad'] = rope.water_angles

ys = {}
lines = {}
handdraws = {}
yax = {}
figs = {}

out_area = ipw.Output()

for quant in z_quants:
    ys[quant] = bq.LinearScale(min=mins[quant], max=maxs[quant])
    lines[quant] = bq.Lines(x=grid.z, y=values[quant], scales={'x': z_scale, 'y': ys[quant]}, 
                            colors=[colors[quant]], interpolation='cardinal')
    handdraws[quant] = bqi.HandDraw(lines=lines[quant])
    yax[quant] = bq.Axis(scale=ys[quant], label=labels[quant], orientation='vertical', grid_lines='none')
    figs[quant] = bq.Figure(marks=[lines[quant]], axes=[z_ax, yax[quant]], interaction=handdraws[quant])
    
    # Update values on handdraw
    # Define the function like this with default argument so that `quant` takes its
    # value at definition time, not evaluation time
    def update_vals(change, quant=quant):
        with out_area:
            print()
            print(quant)
            values[quant] = change['new']
            print('Updated!')
    lines[quant].observe(update_vals, names='y')

links = [
    tr.link((lines['L_mean'], 'y'), (rope, 'kelp_lengths')),
    tr.link((lines['L_std'], 'y'), (rope, 'kelp_stds')),
    tr.link((lines['vw'], 'y'), (rope, 'water_speeds')),
    tr.link((lines['theta_w_rad'], 'y'), (rope, 'water_angles'))
]

In [43]:
ipw.VBox([
    ipw.HBox([
        figs['vw'],
        figs['theta_w_rad'],
    ]),
    ipw.HBox([
        figs['L_mean'],
        figs['L_std']
    ])
])

A Jupyter Widget

# Read + Calculations + Write

In [53]:
write_all(grid, rope, frond, params)

In [ ]:
# Calculate P_kelp

# Visualization

In [ ]:
ipv.quickvolshow(P3D)

In [ ]:
X,Y = np.meshgrid(x,y, indexing='ij')
plt.contour(X,Y,P3D[:,:,0])